In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

%cd /content/gdrive/MyDrive/ThermostabilityProject/
!pip install transformers tqdm

Mounted at /content/gdrive/
/content/gdrive/MyDrive/ThermostabilityProject
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import torch, torch.nn
from transformers import BertModel, BertTokenizer
import re

import re, gc, os
import requests
from tqdm.auto import tqdm

/Users/kvr226/miniconda3/envs/latent_msaprot_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def train_test_validation_splits(df):
    test,train_tot = df.loc[df['set']=='test'],df.loc[df['set']=='train']
    train, val = train_tot.loc[train_tot['validation']!=True], train_tot.loc[train_tot['validation']==True]

    
    return train,val, test

def embed_dataset(dataset_seqs, model, tokenizer, device, shift_left = 1, shift_right = -1):
  inputs_embedding = []

  for sample in tqdm(dataset_seqs):
    with torch.no_grad():
      ids = tokenizer.batch_encode_plus([sample], add_special_tokens=True, padding=True, is_split_into_words=True, return_tensors="pt")
      embedding = model(input_ids=ids['input_ids'].to(device))[0]
      inputs_embedding.append(embedding[0].detach().cpu().numpy()[shift_left:shift_right])

  return torch.from_numpy(np.stack(inputs_embedding))

In [9]:
path_meltome = 'datasets/FLIP/splits/meltome/splits/mixed_split.csv'

splits_meltome = pd.read_csv(path_meltome, sep=',')
train, val, test = train_test_validation_splits(splits_meltome)
print('done')

done


In [10]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
model = model.eval()
if torch.cuda.is_available():
  model = model.half()

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:

dir_pre_generated_embeddings = ''
path_train= dir_pre_generated_embeddings+'train_set_embeddings.pth'
path_test= dir_pre_generated_embeddings+'test_set_embeddings.pth'
path_val= dir_pre_generated_embeddings+'val_set_embeddings.pth'



if os.path.isfile(path_train) and os.path.isfile(path_test) and os.path.isfile(path_val):
    print ("Loading Premade Embeddings...")
    train_embeddings = torch.load(path_train, map_location=device)
    test_embeddings = torch.load(path_test, map_location=device)
    val_embeddings = torch.load(path_val, map_location=device)
    print ("Loaded")
else:
    train_embeddings = embed_dataset(train.sequence, model, tokenizer, device)
    test_embeddings = embed_dataset(test.sequence, model, tokenizer, device)
    val_embeddings = embed_dataset(val.sequence, model, tokenizer, device)
    torch.save(train_embeddings, 'train_set_embeddings.pth')
    torch.save(test_embeddings, 'test_set_embeddings.pth')
    torch.save(val_embeddings, 'val_set_embeddings.pth')

Loading Premade Embeddings...
Loaded


In [16]:
import torch.utils.data as data_utils

batch_size=50
trainDataset = data_utils.TensorDataset(train_embeddings, torch.from_numpy( train.target.to_numpy() ))
testDataset = data_utils.TensorDataset(test_embeddings, torch.from_numpy( test.target.to_numpy() ))
valDataset = data_utils.TensorDataset(val_embeddings, torch.from_numpy( val.target.to_numpy() ))

train_loader = data_utils.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(testDataset, batch_size=batch_size, shuffle=True)
val_loader = data_utils.DataLoader(valDataset, batch_size=batch_size, shuffle=True)

In [ ]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    
    def forward(self, input):
        return input.view(input.size(0), -1)   

class regressionHead(nn.Module):

    def __init__(self, kernel):
        super(regressionHead, self).__init__()
        #self.input_shape = input_shape # pos0 = #channels, pos1 = #diagonal comps, or viseversa

        self.pooling = torch.nn.Sequential(
            torch.nn.AvgPool1d(kernel, stride=2, padding=kernel//2),
            Flatten)

        self.FFNN = torch.nn.Sequential(
            torch.nn.Linear(512, 512),
            torch.nn.LeakyReLU(0.1), #nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.LeakyReLU(0.1), #nn.ReLU(),
            torch.nn.Linear(512, 1)
        )
        
        #self.encoder.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.kaiming_normal_(module.weight)

    def forward(self, x):
        x = self.pooling(x)
        z = self.FFNN(x)
        return z #z.permute(0,2,1) 

In [ ]:
def train_LLMRegresor(train_iterator, val_iterator, model, device, criterion, optimizer, epoch_num):
    
        
        val_loss = []
        
        model.to(device)

        for epoch in range(epoch_num):
            model.train() 
            for i, (input, label) in enumerate(train_iterator):
                optimizer.zero_grad()
                
                input = input.to(device)
                label = label.to(device) 
                out = model(inp)
                loss = criterion(out, label.unsqueeze(-1))
                loss.backward() 
                optimizer.step()

            with torch.no_grad(): # evaluate validation loss here 

                model.eval()
                val_loss_epochs = []

                for (input, label) in val_iterator:
                    
                    input = input.to(device)
                    label = label.to(device)
                    outval = model(input)
                    loss = criterion(o, l.unsqueeze(-1)) # Calculate validation loss 
                    val_loss_epochs.append(loss.item())

                val_loss_epoch = np.mean(val_loss_epochs)
                val_loss.append(round(val_loss_epoch, 3))

            print('epoch: %d loss: %.3f val loss: %.3f' % (epoch + 1, loss.item(), val_loss_epoch))

            # evalutate whether validation loss is dropping; if not, stop
            if epoch > 21:
                if val_loss[-1] >= np.min(val_loss[:-20]): 
                    print('Finished training at epoch {0}'.format(epoch))
                    return epoch 

In [ ]:
model = regressionHead(kernel=5)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss= torch.nn.MSELoss()
epochs = 500
train_LLMRegresor(train_loader, val_loader, model, device, loss, opt, epochs)

In [ ]:



'''sequence_Example = "A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)
print(output)'''

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0454,  0.1140, -0.0117,  ..., -0.0875, -0.1143,  0.0204],
         [ 0.0923,  0.1391, -0.0524,  ..., -0.1395, -0.0428,  0.0743],
         [ 0.1151,  0.0200, -0.0863,  ..., -0.0095, -0.1873,  0.1317],
         ...,
         [ 0.1079,  0.0977, -0.0583,  ..., -0.1277, -0.0649,  0.1289],
         [ 0.0546,  0.0364, -0.0782,  ..., -0.0302, -0.0602,  0.0890],
         [ 0.0515,  0.0571, -0.0693,  ..., -0.0394, -0.0663,  0.0977]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.2487,  0.2626, -0.2367,  ...,  0.2503,  0.2339, -0.2556]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)
